In [1]:
import re

In [2]:
# global variables
obstacles = []
nets = {}
rows = cols = via_cost = non_pref_cost = 0

In [3]:
def parse_input_file(file_path):
    
    with open(file_path) as f:
        global rows, cols, via_cost, non_pref_cost 
        values = f.readline().strip()
        rows, cols, via_cost, non_pref_cost = map(int, values.split(','))

        for line in f:
            line = line.strip()
            if line.startswith("OBS"):
                obstacles.append(tuple(map(int, re.findall(r"\d+", line))))
            elif line.startswith("net"):
                net_name, coords = line.split(' ', 1)
                nets[net_name] = [tuple(map(int, re.findall(r"\d+", coord))) for coord in coords.split(') (')]

In [4]:
def print_input_file_info():
    print(f"Rows: {rows}\nCols: {cols}\nVia Cost: {via_cost}\nNon-preferred Direction Cost: {non_pref_cost}\n")
    
    print("Obstacles:")
    for obs in obstacles:
        print(f"  {obs}")
    
    print("\nNets:")
    for net, coords in nets.items():
        print(f"  {net}: {coords}")

In [5]:
parse_input_file('input.txt')
print_input_file_info()

Rows: 100
Cols: 200
Via Cost: 5
Non-preferred Direction Cost: 10

Obstacles:
  (33, 44)
  (55, 77)

Nets:
  net1: [(1, 10, 20), (2, 30, 50), (1, 5, 100)]
  net2: [(2, 100, 200), (1, 300, 50)]
  net3: [(1, 100, 50), (2, 300, 150), (2, 50, 50), (1, 2, 2)]
